# OpenET SSEBop
## Single Landsat Collection 1 TOA Image Example

In [1]:
import pprint

import ee
from IPython.display import Image

import openet.ssebop

ee.Initialize()

In [3]:
ndvi_palette = ['#EFE7E1', '#003300']
et_palette = [
  'DEC29B', 'E6CDA1', 'EDD9A6', 'F5E4A9', 'FFF4AD', 'C3E683', '6BCC5C', 
  '3BB369', '20998F', '1C8691', '16678A', '114982', '0B2C7A']

### Landsat 8 Collection 1 TOA Image

In [4]:
landsat_img = ee.Image('LANDSAT/LC08/C01/T1_RT_TOA/LC08_044033_20170716')
# landsat_img = ee.Image('LANDSAT/LC08/C01/T1_RT_TOA/LC08_044033_20170801')
Image(url=landsat_img.select([3, 2, 1]).getThumbURL({'min': 0.0, 'max': 0.3}))

In [5]:
landsat_cs = 30
landsat_crs = landsat_img.select('B3').projection().getInfo()['crs']
landsat_region = landsat_img.geometry().bounds(1, 'EPSG:4326').coordinates().getInfo()
landsat_dt = ee.Date(landsat_img.get('system:time_start'))

In [6]:
# Build the SSEBop object
ssebop_obj = openet.ssebop.Image.from_landsat_c1_toa(landsat_img)

### NDVI

In [7]:
Image(url=ssebop_obj.ndvi.getThumbURL({'min': -0.1, 'max': 0.9, 'palette': ','.join(ndvi_palette)}))

### ET fraction

In [8]:
Image(url=ssebop_obj.etf.getThumbURL({'min': 0.0, 'max': 1.2, 'palette': ','.join(et_palette)}))

### GRIDMET Daily Reference ET

In [9]:
etr_img = ee.Image(ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').select(['etr'])\
    .filterDate(landsat_dt, landsat_dt.advance(1, 'day'))\
    .first())

# Clip and project to the Landsat image footprint and coordinate system
Image(url=etr_img\
          .reproject(crs=landsat_crs, scale=landsat_cs)\
          .getThumbURL({'min': 0.0, 'max': 15, 'palette': ','.join(et_palette), 
                        'region': landsat_region}))

### Daily ET

ET = ETf * ETr

In [10]:
et_img = ssebop_obj.etf.multiply(etr_img)
Image(url=et_img.getThumbURL({'min': 0.0, 'max': 15, 'palette': ','.join(et_palette)}))

### SSEBop Image Metadata

In [11]:
pprint.pprint(ssebop_obj.etf.getInfo()['properties'])

{'TCORR': 0.992556758029887,
 'TCORR_INDEX': 0,
 'system:index': 'LC08_044033_20170716',
 'system:time_start': 1500230731090}
